In [65]:
import torch
import torch.nn as nn
import numpy as np
from sklearn import datasets
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import pandas as pd

In [66]:
# prepare data
bc = datasets.load_breast_cancer()

# convert to pandas dataframe - Not necessary
df = pd.DataFrame(
    data=np.c_[bc.data, bc.target], 
    columns=np.append(bc['feature_names'], ['target']))

print(df.shape)

X, y = bc.data, bc.target

n_samples, n_features = X.shape

# Split data into train en test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1234)

# Scale features
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.fit_transform(X_test)

# Convert to tensors
X_train = torch.from_numpy(X_train.astype(np.float32))
X_test = torch.from_numpy(X_test.astype(np.float32))
y_train = torch.from_numpy(y_train.astype(np.float32))
y_test = torch.from_numpy(y_test.astype(np.float32))

# Reshape Y tensor -> has one row now
y_train = y_train.view(y_train.shape[0], 1)
y_test = y_test.view(y_test.shape[0], 1)

(569, 31)


In [67]:
# Create model
class LR(nn.Module):
    def __init__(self, n_input_features):
        super(LR, self).__init__()
        # Define amount of input features and output features
        self.linear = nn.Linear(n_input_features, 1)
    
    def forward(self, x):
        y_pred = torch.sigmoid(self.linear(x))
        return y_pred

model = LR(n_features)

In [68]:
# set loss and optimizer
learning_rate = 0.01
criterion = nn.BCELoss()
optimizer = torch.optim.SGD(model.parameters(), learning_rate)

In [72]:
# Training
n_epochs = 100

for epoch in range(n_epochs):
    # Forward pass
    y_pred = model(X_train)
    loss = criterion(y_pred, y_train)

    # Backward pass
    loss.backward()

    # Update weights
    optimizer.step()

    # Empty gradients
    optimizer.zero_grad()

    if (epoch+1) % 10 ==0:
        print(f"epoch: {epoch+1}, loss: {loss.item():.4f}")

epoch: 10, loss: 0.5461
epoch: 20, loss: 0.4467
epoch: 30, loss: 0.3853
epoch: 40, loss: 0.3436
epoch: 50, loss: 0.3131
epoch: 60, loss: 0.2897
epoch: 70, loss: 0.2709
epoch: 80, loss: 0.2555
epoch: 90, loss: 0.2426
epoch: 100, loss: 0.2315


In [76]:
# Acc
with torch.no_grad():
    y_pred = model(X_test)
    y_pred_cls = y_pred.round()
    acc = y_pred_cls.eq(y_test).sum() / float(y_test.shape[0])

    print(f'Accuracy: {acc:.4f}')

Accuracy: 0.9211
